## Doctor Right


#### Import Library

In [17]:
%pip install keras
%pip install optree
%pip install tensorflow
%restart_python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%restart_python` not found.


In [2]:
import sys
sys.path.append("../modules")
from eda import EDAAnalyzer
from spark_session import SparkManager
from feature_engineering import FeatureEngineer

In [3]:
# Load autoreload extension
%load_ext autoreload
%autoreload 2

#### Constants and config

In [4]:
mx_submits_path = "../data_sample/mx_submits.parquet/"
mx_submits_line_path = "../data_sample/mx_submitsline.parquet/"
cohort_key="767ef4cac69e8a0c77384f6e1414364b"

sample_patient_id = "8aad41f612a7095449888c8050abaeb05fdee65643caa3033542610421d8bd1daaa2c4ce1757401003a1bbcd60948a7aa13eba507a676dea80e0cf76b77dbc95"
features_cols = [
# 'secondary_payer_state',
# 'facility_provider_address_region',
'patient_gender',
# 'patient_location_residential_region',
# 'principal_diagnosis_body_part',
# 'principal_diagnosis_category',
# 'claim_all_diagnosis_codes',
'claim_total_charge_amount',
'previous_diagnosis_ohe']
exclude_cols = ['patient_id']

most_reapeated_diagnosis_5char = [    'Z0000', 'Z1211', 'R0602', 'Z1231', 'R5383', 'R0789', 'K5900', 'G8929', 'G4733', 'I2510',    'E7800', 'R0981', 'Z7189', 'J0190', 'Z1159', 'K5730', 'Z7182', 'Z1389', 'Z0001', 'R9431',    'E6601', 'R1013', 'M6281', 'R1084', 'R7989', 'Z6852', 'Z1331', 'H5213', 'M5450', 'R0600',    'G4700', 'H2513', 'M1990', 'M7989', 'E1165', 'R1110', 'M5416', 'Z7982', 'R4182', 'I4891',    'M5136', 'R0989', 'R1310', 'R1031', 'R7303', 'R6889', 'M1711', 'K2970', 'R1011', 'H5203',    'R1032', 'Z7901', '25000', 'Z8249', 'R5381', 'M1712', 'R1030', 'G8918', 'R0902', 'J0100',    'R7309', 'R2689', 'J3089', 'H6123', 'Z6841', 'Z7984', 'Z0100', 'M5412', 'J3489', 'Z5181',    'J4520', 'Z9181', 'J9601', 'Z1152', 'M2550', 'Z3202', 'Z1151', 'G4730', 'Z6830', 'H6691',    'Z0110', 'Z6829', 'Z6831', 'J9811', 'Z6828', 'H2512', 'Z6827', 'H6692', 'R0683', 'Z1329',    'H2511', 'M9903', 'K3189', 'Z7689', 'Z6854', 'N3000', 'Z1239', 'R7301', 'M9902', 'V7231',    'Z6825', 'Z6832', 'Z6826', 'M5126', 'M9901', 'M7910', 'Z3009', 'K8020', '53081', 'E1122',    'K2950', 'Z0189', 'Z9049', 'B9689', '78079', 'V7612', 'Z6833', 'R0609', 'F0390', 'V5869',    'E6609', 'Z6853', 'Z6834', 'M8580', 'H6693', 'Z8673', 'Z6835', 'M4802', '78650', 'Z6824',    'Z6823', 'E1142', 'R0781', 'Z4789', 'R0982', 'F4310', 'I6523', 'D1801', 'B9789', 'E1169',    'Z8719', 'Z3A39', 'E1140', 'Z3800', 'Z6836', 'F1120', 'K6389', '30000', 'J0390', 'K5909',    'M5441', 'H9202', 'H9201', 'R2681', 'F1010', '78900', 'R3915', 'Z6837', 'R9720', 'Z8616',    '78605', 'Z3201', 'N3001', 'Z2821', 'Z0389', 'Z9989', '49390', 'N1830', 'E8342', 'M5442',    'R1012', 'R5382', 'F4323', 'H6121', 'K2100', 'Z3483', 'R8290', '27800', 'R0689', 'Z6822',    'M5137', 'Z6842', 'H6690', 'Z1388', 'L0390', 'M7918', 'K7689', 'Z6838', 'K5790', 'M4316',    'I5032', '71946', 'H1033', 'M5116', 'Z3A36', 'M1611', 'G5601', 'Z3A37', 'F1020', 'Z8679',    '32723', 'H9203', 'Z4802', 'Z3A38', 'Z3A01', 'E7849', 'Z8619', 'N1831', 'K4090', 'Z4682',    'G9340', '78060', 'M5417', 'E1151', 'K2900', 'R9389', 'J4530', 'G9341', 'Z9981', 'R1010',    'V0481', 'Z1212', 'R3129', 'H6122', 'K6289', 'M1612', 'J0110', 'K0889', 'Z6839', 'M7541',    'R1033', 'M8589', 'Z6821', 'Z3482', 'M5030', 'F4321', 'H6983', 'N6489', 'M4726', 'H1013',    'Z0289', 'Z5189', 'Z7951', 'Z3A00', 'J9600', 'Z7902', 'Z9189', 'J4540', 'M5459', 'Z0101',    'F4320', 'F5101', 'Z3041', 'N1330', 'Z3A35', 'L0100', 'Z3481', 'I5022', '41401', 'G2581',    'H9190', 'G5602', 'M9904', 'N3941', 'H9313', 'Z1283', 'V0382', 'I2720', 'H1045', 'N2889',    'M9905', 'Z1339', 'M8588', 'N3281', 'Z4889', 'M5127', 'V7651', 'E8352', 'Z1321', 'M4806',    'H9193', 'J3081', 'N1832', 'Z3490', 'Z3689', 'J9801', 'G9389', '41400', 'Z3A32', 'Z0130',    'Z6820', 'R1312', 'F1210', 'M5431', 'Z8669', 'Z3A20', '71941', 'G5603', 'F4322', 'R7401',    'Z3A34', 'R6883', 'I4510', 'E1159', 'I2699', 'Z6843', '78609', 'M7542', '42731', 'Z7251',    '56400', 'G4710', 'R7881', 'F1290', 'J9690', 'H9209', '78791', 'K5901']



### Spark Session

In [5]:
mx_submits_spark_manager = SparkManager(mx_submits_path)
# mx_submits_spark_manager = SparkManager(cohort_key=cohort_key)
# mx_submits_line_spark_manager = SparkManager(mx_submits_line_path)

24/10/17 23:19:44 WARN Utils: Your hostname, Sureshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.9.214 instead (on interface en0)
24/10/17 23:19:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/17 23:19:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### MX SUBMITS

#### EDA

In [ ]:
mx_submits_eda = EDAAnalyzer(mx_submits_spark_manager)

In [ ]:
mx_submits_eda.display_head()

In [ ]:
mx_submits_eda.display_shape()

#### Type conversion

In [ ]:
mx_submits_eda.convert_columns_to_float(["claim_total_charge_amount"])

In [ ]:
column_info_submits = mx_submits_eda.display_column_info()
column_info_submits.to_csv("../output/column_info_submits.csv")
column_info_submits

In [ ]:
mx_submits_eda.plot_percentile_based_cutoff("claim_all_diagnosis_codes",90,cutoff_length=5, per_patient= True)

In [ ]:
mx_submits_claim_all_diagnosis_codes_repeat_count = mx_submits_eda.get_top_n_repeated_values_unique_per_patient("claim_all_diagnosis_codes")
mx_submits_claim_all_diagnosis_codes_repeat_count

In [ ]:
most_repeated_diagnosis = mx_submits_claim_all_diagnosis_codes_repeat_count[
    (mx_submits_claim_all_diagnosis_codes_repeat_count["diagnosis_code_length"] == 5) & 
    (mx_submits_claim_all_diagnosis_codes_repeat_count["count"] > 248)
]
most_reapeated_diagnosis_5char = most_repeated_diagnosis['diagnosis_code'].tolist()
most_repeated_diagnosis

In [ ]:
most_reapeated_diagnosis_5char

 - (ICD10, Z0001, 1): Encounter for general adult medical examination with abnormal findings
 - (ICD10, F4323, 1): Adjustment disorder with depressed mood
 - (ICD10, Z452, 1)	: Pneumonia, unspecified organism
 - (ICD10, J441, 1)	: Chronic obstructive pulmonary disease with acute exacerbation

In [ ]:
mx_submits_eda.get_top_n_repeated_values("claim_all_diagnosis_codes",5)

### Feature Engineering

In [15]:
mx_submits_fe=FeatureEngineer(mx_submits_spark_manager)

In [16]:
mx_submits_fe.truncate_diagnosis_codes()

In [7]:
# mx_submits_fe.transform_claim_all_diagnosis_codes()

In [8]:
mx_submits_fe.add_comorbidities_with_exponential_decay_sparse_vector()

,previous_diagnosis_ohe
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [24]:
mx_submits_fe.dataframe.select("previous_diagnosis_ohe").take(5)[0]["previous_diagnosis_ohe"]

SparseVector(13659, {})

In [9]:
mx_submits_fe.remove_diagnosis_codes(most_reapeated_diagnosis_5char)

In [10]:
mx_submits_fe.retain_columns(features_cols)

In [11]:
# mx_submits_fe.convert_columns_to_float(["claim_total_charge_amount"])
preprocess_data = mx_submits_fe.preprocess_data(features_cols)
preprocess_data

Assembling all features into a vector with 1 columns.
Preprocessing complete. Feature vector created.


In [62]:
mx_submits_fe.dataframe.select("features").take(1)[0]["features"][0].size

13659

In [67]:
mx_submits_fe.dataframe.head()["features"][0].size

13659

In [12]:
mx_submits_fe.reduce_dataframe_size(1000)
mx_submits_fe.train_autoencoder()
# mx_submits_fe.save_autoencoder("../output/model")

24/10/17 23:20:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Approximate distinct count: 1000
Steps per epoch: 3
Epoch 1/50


2024-10-17 23:45:58.188063: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


3/3 - 5s - loss: 0.1249 - 5s/epoch - 2s/step
Epoch 2/50


...

3/3 - 4s - loss: 0.0160 - 4s/epoch - 1s/step
Epoch 46/50


3/3 - 4s - loss: 0.0144 - 4s/epoch - 1s/step
Epoch 47/50


3/3 - 4s - loss: 0.0138 - 4s/epoch - 1s/step
Epoch 48/50
3/3 - 1s - loss: 0.0122 - 673ms/epoch - 224ms/step
Epoch 49/50


3/3 - 4s - loss: 0.0115 - 4s/epoch - 1s/step
Epoch 50/50


3/3 - 3s - loss: 0.0104 - 3s/epoch - 1s/step


In [ ]:
mx_submits_fe.load_autoencoder("../output/model")

In [ ]:
mx_submits_fe.create_feature_name_map()

In [ ]:
feature_impact = mx_submits_fe.evaluate_feature_impact()
feature_impact

In [ ]:
feature_impact["Feature Name"].to_list()[:300]